In [6]:
!pip install tree_sitter
!git clone https://github.com/tree-sitter/tree-sitter-python /workspace/tmp/code2ast-nmt/tree-sitter-python

Cloning into '/workspace/tmp/code2ast-nmt/tree-sitter-python'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 1989 (delta 15), reused 26 (delta 7), pack-reused 1949
Receiving objects: 100% (1989/1989), 17.50 MiB | 1.64 MiB/s, done.
Resolving deltas: 100% (1271/1271), done.


In [2]:
from tree_sitter import Language, Parser, Node, TreeCursor
from collections import deque

import typing

In [3]:
def find_substring_node(
    program_lines: typing.List[str], 
    node: Node
) -> str:
    start = node.start_point
    end = node.end_point
    return find_substring(program_lines, start, end)

def find_substring(
    program_lines: typing.List[str], 
    start: typing.Tuple[int, int], 
    end: typing.Tuple[int, int]
) -> str:
    lines: typing.List[str] = []
        
    for idx in range(start[0], end[0] + 1):
        _start = 0
        _end = len(program_lines[idx])
        
        if idx == start[0]:
            _start = start[1]
        if idx == end[0]:
            _end = end[1]
        
        lines.append(program_lines[idx][_start:_end])
        
    return lines


def traverse_deque(node: Node) -> Node:
    d = deque(node.children)
    while d:
        n = d.popleft()
        # d.extend(n.children)
        if n.is_named: 
            d.extendleft(n.children[::-1])
            yield n

In [4]:
Language.build_library(
  output_path="/workspace/tmp/code2ast-nmt/build/my-languages.so",
  repo_paths=[
    "/workspace/tmp/code2ast-nmt/tree-sitter-python"
  ]
)

False

In [5]:
PY_LANGUAGE = Language("/workspace/tmp/code2ast-nmt/build/my-languages.so", "python")

In [6]:
parser = Parser()
parser.set_language(PY_LANGUAGE)

In [7]:
program = """\
def hello_world(name: str) -> None:
    print(f"Hello World by {name}!")\
"""

program_lines = program.split("\n")
tree = parser.parse(bytes(program, "utf8"))

In [209]:
program_lines

['def hello_world(name: str) -> None:', '    print(f"Hello World by {name}!")']

In [8]:
def exec2():
    r = ""
    for x in traverse_deque(tree.root_node):
        if x.type == "identifier":
            r += "".join(find_substring_node(program_lines, x)) + " "
        else:
            r += str(x.type) + " "
    return r

In [9]:
%%time
exec2()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 151 µs


'function_definition hello_world parameters typed_parameter name type str type none block expression_statement call print argument_list string interpolation name '